In [ ]:
import os
import csv
import cv2
import math
import time
import json
import glob
import shutil
import seaborn as sn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm as tqdm
from tabulate import tabulate

In [ ]:
experiment_name = "0908_164_set_60"
# 用以區別 csv 檔名稱
Resave = False
# 避免覆蓋，確定要覆蓋再改 true
label_all_crown = './result/seg_raw/*.PNG'
# 放 Unet_test.ipynb 跑出來的 label
label_all_crown_list = glob.glob(label_all_crown)
label_all_crown_list.sort()
print(len(label_all_crown_list))

In [ ]:
pred_all_crown = './label/visual_label/*.PNG'
# 放 test 資料夾裡原本給的 label
pred_all_crown_list = glob.glob(pred_all_crown)
pred_all_crown_list.sort()
print(len(pred_all_crown_list))

In [ ]:
def show_pixel_set(img_nparray):
    a = img_nparray
    unique, counts = np.unique(a, return_counts=True)
    return dict(zip(unique, counts))

In [ ]:
def count_iou(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    intersection = np.logical_and(label_target, predict_target)
    union = np.logical_or(label_target, predict_target)
    iou_score = np.sum(intersection) / np.sum(union)
#     print('IOU val:',iou_score)
    return iou_score

In [ ]:
def show_iou(label_img, predict_img, row_contents):
    label_img_gray = cv2.cvtColor(label_img, cv2.COLOR_BGR2GRAY)
    predict_img_gray = cv2.cvtColor(predict_img, cv2.COLOR_BGR2GRAY)
    gray_pixel_dict = {'img_background': 0, 'img_gums': 29,
                     'img_cavity': 76, 'img_bacmixgums': 86,
                     'img_overlap': 105, 'img_cej': 150,
                     'img_depressed': 151, 'img_artifical_crown': 179,
                     'img_tooth': 226, 'img_whiteside': 255}
    for target in gray_pixel_dict:
#         print('class {}:'.format(target))
        IOU_scale = count_iou(label_img_gray, predict_img_gray, gray_pixel_dict[target])
        row_contents.append(IOU_scale)
    return row_contents

In [ ]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = csv.writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
#initial csv file
csv_file_path = './IOU_csv/{}_IOU_result.csv'.format(experiment_name)

assert not os.path.exists(csv_file_path) or Resave, "This csv file is exist,if want to resave please modify Resave parameter."

print('Initial the csv.')
with open(csv_file_path, 'w', newline='\n') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image_name', 'background IOU', 'gums IOU', 'cavity IOU', 'bacmixgums IOU',
                         'overlap IOU','cej IOU','depressed IOU','artifical_crown IOU','tooth IOU',
                        'whiteside IOU'])

In [ ]:
for all_crown in tqdm(label_all_crown_list, total = len(label_all_crown_list)):
    row_contents = []
    img_name = os.path.basename(all_crown)
    pre_target = [img_path for img_path in pred_all_crown_list if img_name in img_path]
#     shutil.copyfile(pre_target[0], os.path.join(save_pp,img_name))
#     o_img_target = [img_path for img_path in o_img_list if img_name in img_path]
#     o_img = cv2.imread(o_img_target[0])
    if pre_target !=[]:
        label_img = cv2.imread(all_crown)
        pre_img = cv2.imread(pre_target[0])

        ##IOU fumction
        row_contents.append(img_name)
        row_contents_all = show_iou(label_img,pre_img,row_contents)
        append_list_as_row(csv_file_path, row_contents_all)

In [ ]:
csv_file = pd.read_csv(csv_file_path)

background_list = list(csv_file['background IOU'].dropna())
background_list = [ele for ele in background_list if ele > 0]
background = sum(background_list)/len(background_list)

gums_list = list(csv_file['gums IOU'].dropna())
gums_list = [ele for ele in gums_list if ele > 0]
gums = sum(gums_list)/len(gums_list)

cavity_list = list(csv_file['cavity IOU'].dropna())
cavity_list = [ele for ele in cavity_list if ele > 0]
cavity = sum(cavity_list)/len(cavity_list)

bacmixgums_list = list(csv_file['bacmixgums IOU'].dropna())
bacmixgums_list = [ele for ele in bacmixgums_list if ele > 0]
bacmixgums = sum(bacmixgums_list)/len(bacmixgums_list)


overlap_list = list(csv_file['overlap IOU'].dropna())
overlap_list = [ele for ele in overlap_list if ele > 0]
overlap = sum(overlap_list)/len(overlap_list)

cej_list = list(csv_file['cej IOU'].dropna())
cej_list = [ele for ele in cej_list if ele > 0]
cej = sum(cej_list)/len(cej_list)

depressed_list = list(csv_file['depressed IOU'].dropna())
depressed_list = [ele for ele in depressed_list if ele > 0]
depressed = sum(depressed_list)/len(depressed_list)

artifical_crown_list = list(csv_file['artifical_crown IOU'].dropna())
artifical_crown_list = [ele for ele in artifical_crown_list if ele > 0]
artifical_crown = sum(artifical_crown_list)/len(artifical_crown_list)

tooth_list = list(csv_file['tooth IOU'].dropna())
tooth_list = [ele for ele in tooth_list if ele > 0]
tooth = sum(tooth_list)/len(tooth_list)

whiteside_list = list(csv_file['whiteside IOU'].dropna())
whiteside_list = [ele for ele in whiteside_list if ele > 0]
whiteside = sum(whiteside_list)/len(whiteside_list)

row_contents = ['Mean', background, gums, cavity, bacmixgums, overlap, cej, depressed, artifical_crown, tooth,whiteside]
append_list_as_row(csv_file_path, row_contents)

In [ ]:
print('1 牙冠',cej*100,'%')
print('2 人造牙冠與補牙',artifical_crown*100,'%')
print('3 健康牙周骨頭',gums*100,'%')
print('4 有骨質流失牙周骨頭',depressed*100,'%')
print('5 象牙質',tooth*100,'%')
print('6 牙隨腔',cavity*100,'%')
print('7 重疊區',overlap*100,'%')
print('8 背景模糊',bacmixgums*100,'%')
print('9 背景',background*100,'%')
print('10 白框',whiteside*100,'%')
print('Sum',(cej+artifical_crown+gums+depressed+tooth+cavity+overlap+bacmixgums+background+whiteside)*100,'%')